In [47]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [48]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [49]:
def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [50]:
def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    

In [51]:
def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [52]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2016-10-21 00:00:00")

print(len(training_set))

# obtengo los nodos del grafo:
nodes = training_set.problem_id.unique()


# creo un diccionario que va a tener a los problemas como keys y los valores seran los
# usuarios que han hecho ese problema
grouped = training_set.groupby('problem_id')['user_id'].apply(list)

print(len(nodes))
# print(grouped)


10262
169


In [53]:
print(training_set)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [54]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2016-10-21 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
16939         469      799     RF  2016-10-21 00:08:23
16940         325     3832     AC  2016-10-21 00:08:35
16942         469     3757     AC  2016-10-21 00:41:36
16943         469     3792    RTE  2016-10-21 01:04:08
16944         469      810     AC  2016-10-21 01:48:35
16945         438     3943     AC  2016-10-21 02:23:56
16946         469     3773     AC  2016-10-21 05:00:58
16947         346     3728     AC  2016-10-21 09:06:38
16948         469     3736     CE  2016-10-21 09:38:14
16949         438     3785     TL  2016-10-21 09:43:52
16950         469     3704     AC  2016-10-21 10:04:03
16951         469     2912     AC  2016-10-21 10:20:36
16952         213     3136     AC  2016-10-21 10:28:23
16953         213     3088     AC  2016-10-21 10:29:26
16954         469     3708    RTE  2016-10-21 10:35:15
16956         235     3136    RTE  2016-10-21 10:37:58
16957         183     3880     AC  2016-10-21 11:38:11
16958     

In [55]:

# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

10837


In [56]:
def apply_jn(row, graph):
    """
        Funcion que devuelve el valor de jaccard coefficient
    """
    values_jn = nx.jaccard_coefficient(graph, [(row['one'], row['two'])])
    
    value_jn = 0
    for u, v, p in values_jn:
        value_jn = p # saco el valor
        
    return value_jn

def create_jn_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de jn para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_jn = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_jn['jn'] = dataFrame_jn.apply (lambda row: apply_jn(row, graph), axis=1)


    return dataFrame_jn

In [57]:
jn_df = create_jn_data(graph, nodes)
print(jn_df)

       one  two        jn
0       10    2  0.948052
1       10    4  0.966887
2       10    6  0.966887
3       10   13  0.953333
4       10   15  0.928571
5       10   39  0.948052
6       10   33  0.947712
7       10   44  0.966443
8       10   19  0.933333
9       10   27  0.960526
10      10   60  0.817568
11      10  100  0.948052
12      10   93  0.358108
13      10   53  0.921053
14      10   51  0.952703
15      10   49  0.966667
16      10   70  0.939597
17      10   62  0.953642
18      10   35  0.933775
19      10   81  0.871622
20      10    8  0.879195
21      10   17  0.810811
22      10   23  0.939597
23      10   25  0.000000
24      10   86  0.954248
25      10   29  0.925676
26      10   31  0.000000
27      10   47  0.000000
28      10   65  0.934641
29      10   73  0.906040
...    ...  ...       ...
14166  510  509  0.953333
14167  510  507  0.707483
14168  511  504  0.000000
14169  511  503  0.941176
14170  511  505  0.940000
14171  511  506  0.906040
14172  511  

In [58]:
# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user = training_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_problem_id':grouped_user.values})

print(df_users)

                                        list_problem_id  user_id
0                                     [10, 76, 489, 39]        5
1                                              [2, 487]        6
2                      [39, 44, 76, 105, 109, 114, 117]        7
3     [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4          [2, 10, 4, 6, 13, 253, 57, 62, 143, 83, 109]        9
5                                             [307, 39]       10
6                           [33, 109, 39, 93, 100, 155]       11
7     [13, 33, 39, 44, 100, 51, 49, 70, 81, 53, 130,...       12
8                                         [44, 187, 39]       15
9                                                  [39]       16
10                                             [39, 86]       17
11                            [119, 114, 316, 507, 506]       18
12                                                 [44]       22
13                                            [100, 13]       24
14    [258, 141, 309, 310

In [59]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    
    

In [60]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)


      user_id  len_training  len_evaluation
0           5             4               1
1           6             2               0
2           9            11               3
3           8            47               0
4          16             1               0
5          17             2               2
6          12            12               0
7          18             5               8
8          15             3               1
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            41               1
16         33             9               0
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [61]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))


      user_id  len_training  len_evaluation
7          18             5               8
29         60            18              14
118        25            22               7
261       130            80               6
299       414            49              61
316       443            53               8
317       448            43              21
483       689            33               7
619       912            13              11
633       935           127               9
1196     1711           128              16
1288     1893             7               6
1311     1952            19               9
1339     1955             9               5
1416     2038             5               5
1423     2096            12              13
1446     2051            11               8
1447     2025            12               8
1448     2120             7               8
1451     2041             9               7
1528     2257             6              13
1694     2576             5     

In [62]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users)

                                        list_problem_id  user_id
11                            [119, 114, 316, 507, 506]       18
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711
997                      [39, 49, 465, 159, 134, 13, 2]     1893
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952
1024      [325, 327, 441, 468, 471, 503, 226, 109, 446]     1955
1066  [155, 505, 49, 162,

In [63]:
# creo el nuevo dataframe con los resultados 
column_problem_recomend = {'problem_id': nodes}
dataframe_problem_recomend = pd.DataFrame.from_dict(column_problem_recomend)

print(dataframe_problem_recomend)

     problem_id
0            10
1             2
2             4
3             6
4            13
5            15
6            39
7            33
8            44
9            19
10           27
11           60
12          100
13           93
14           53
15           51
16           49
17           70
18           62
19           35
20           81
21            8
22           17
23           23
24           25
25           86
26           29
27           31
28           47
29           65
..          ...
139         444
140         437
141         438
142         443
143         445
144         442
145         440
146         446
147         465
148         471
149         468
150         470
151         469
152         467
153         489
154         485
155         486
156         487
157         484
158         488
159         508
160         510
161         511
162         504
163         503
164         505
165         506
166         502
167         509
168         507

[169 ro

In [64]:
def getSimilarProblems(row, jn_df):
    """
        Funcion que devuelve la lista de los problemas similares del problema de la fila: aquellos que tienen un valor jn
        mayor a cero
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = jn_df[jn_df['one'] == row['problem_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['jn'] > 0]
    column_result_two_tmp = jn_df[jn_df['two'] == row['problem_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['jn'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_jn = list_one + list_two
    
    # print(list_jn)
    
    if list_jn == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_jn = graph.nodes()
        list_jn.remove(row['problem_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_jn

In [65]:
dataframe_problem_recomend['neighbors'] = dataframe_problem_recomend.apply (lambda row: getSimilarProblems(row, jn_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_problem_recomend)

     problem_id                                          neighbors
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...
13           93  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 2

In [66]:
print(grouped)
# convierto la serie en un dataframe
df_users_recommend = pd.DataFrame({'problem_id':grouped.index, 'list_user_id':grouped.values})
print(df_users_recommend)

problem_id
2      [6, 9, 33, 67, 56, 84, 35, 60, 112, 124, 106, ...
4      [9, 8, 35, 84, 126, 44, 39, 313, 42, 437, 130,...
6      [9, 46, 35, 67, 89, 84, 137, 152, 231, 221, 88...
8      [35, 52, 133, 129, 130, 437, 511, 605, 935, 67...
10     [5, 9, 35, 133, 129, 80, 137, 247, 437, 103, 5...
13     [9, 12, 59, 35, 67, 109, 54, 24, 55, 106, 136,...
15     [8, 35, 67, 55, 133, 129, 84, 167, 205, 231, 2...
17           [35, 133, 129, 437, 103, 206, 62, 89, 1711]
19     [8, 35, 44, 205, 437, 527, 448, 1063, 206, 62,...
23     [35, 67, 60, 437, 206, 62, 103, 864, 1001, 113...
25                                        [35, 103, 130]
27     [8, 35, 67, 167, 39, 440, 448, 437, 130, 488, ...
29     [35, 133, 86, 112, 42, 232, 248, 247, 314, 320...
31                                        [35, 437, 448]
33     [12, 59, 35, 84, 111, 96, 152, 126, 155, 30, 1...
35     [64, 35, 214, 120, 236, 82, 409, 390, 175, 249...
39     [16, 17, 12, 7, 31, 40, 33, 66, 68, 71, 69, 70...
44     [12, 15, 22, 

In [67]:
def getUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de usuarios que han
        realizado los problemas similares al de la fila. Además eliminara los usuarios que ya hayan realizado el problema.
    """
    
    # obtengo la lista de usuarios que han hecho el problema en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # print(list_problems_user)

    # lista resultante de la concatenacion de las listas de usuarios de los problemas similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de usuarios que han hecho el problema similar
        list_problems_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_user_id'])[0]
        
        # print("----------")
        #print(lista_problemas_comprobar)
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_problems
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [68]:
# ahora para cada problema, hacer una lista de los usuarios similares a los que han realizado ese problema, 
# que no sean usuarios que han realizado ya ese problema
dataframe_problem_recomend['list_users'] = dataframe_problem_recomend.apply (lambda row: getUsersFromSimilarProblems(row, df_users_recommend), axis=1)
print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [69]:
# elimino la columna de los problemas vecinos, ya que no interesa
del dataframe_problem_recomend['neighbors']

print(dataframe_problem_recomend)

     problem_id                                         list_users
0            10  [6, 33, 67, 56, 84, 60, 112, 124, 106, 41, 158...
1             2  [126, 39, 313, 42, 1902, 1983, 1959, 1988, 195...
2             4  [46, 67, 137, 152, 231, 221, 88, 194, 205, 90,...
3             6  [12, 59, 109, 54, 24, 55, 106, 136, 82, 42, 13...
4            13  [167, 205, 231, 224, 41, 452, 437, 448, 665, 8...
5            15  [16, 17, 12, 7, 40, 33, 66, 68, 71, 69, 70, 65...
6            39  [59, 152, 196, 271, 274, 269, 270, 272, 277, 2...
7            33  [15, 22, 7, 33, 129, 117, 202, 156, 54, 153, 2...
8            44  [8, 44, 205, 437, 527, 448, 1063, 1542, 1480, ...
9            19  [67, 167, 39, 440, 130, 488, 1261, 1130, 1603,...
10           27  [53, 84, 114, 55, 232, 133, 47, 191, 431, 1441...
11           60  [28, 29, 12, 32, 33, 49, 42, 46, 24, 60, 59, 5...
12          100  [91, 1500, 1659, 1922, 1799, 571, 8, 67, 31, 1...
13           93  [8, 12, 67, 133, 129, 60, 31, 189, 383, 206, 

In [70]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_users'] if not (x in conjunto_vacio or function_add(x))]

In [71]:

# tengo que contar el numero de apariciones de cada usuario en la lista de usuarios

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de usuarios sin repeticiones
dataframe_problem_recomend['lista_users_unique'] = dataframe_problem_recomend.apply(lambda row: delRepetitions(row), axis=1)

print(dataframe_problem_recomend)

     problem_id                                         list_users  \
0            10  [6, 33, 67, 56, 84, 60, 112, 124, 106, 41, 158...   
1             2  [126, 39, 313, 42, 1902, 1983, 1959, 1988, 195...   
2             4  [46, 67, 137, 152, 231, 221, 88, 194, 205, 90,...   
3             6  [12, 59, 109, 54, 24, 55, 106, 136, 82, 42, 13...   
4            13  [167, 205, 231, 224, 41, 452, 437, 448, 665, 8...   
5            15  [16, 17, 12, 7, 40, 33, 66, 68, 71, 69, 70, 65...   
6            39  [59, 152, 196, 271, 274, 269, 270, 272, 277, 2...   
7            33  [15, 22, 7, 33, 129, 117, 202, 156, 54, 153, 2...   
8            44  [8, 44, 205, 437, 527, 448, 1063, 1542, 1480, ...   
9            19  [67, 167, 39, 440, 130, 488, 1261, 1130, 1603,...   
10           27  [53, 84, 114, 55, 232, 133, 47, 191, 431, 1441...   
11           60  [28, 29, 12, 32, 33, 49, 42, 46, 24, 60, 59, 5...   
12          100  [91, 1500, 1659, 1922, 1799, 571, 8, 67, 31, 1...   
13           93  [8,

In [72]:
# ahora voy a separar cada problema_user para hacer la cuenta
# creo un nuevo dataframe que agrupa por el problema con su posible usuario
df_separation = dataframe_problem_recomend.groupby(['problem_id']).lista_users_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation.columns = ['problem_id', 'possible_user']

print(df_separation)

        problem_id  possible_user
0                2            126
1                2             39
2                2            313
3                2             42
4                2           1902
5                2           1983
6                2           1959
7                2           1988
8                2           1952
9                2           1962
10               2           3497
11               2             46
12               2            152
13               2            221
14               2             88
15               2            194
16               2             90
17               2             99
18               2            155
19               2            542
20               2            896
21               2           1186
22               2            598
23               2            584
24               2            596
25               2            594
26               2            593
27               2            581
28            

In [73]:
def getScore(row, dataframe_problem_recomend):
    """
        Funcion que devuelve el numero de apariciones del posible usuario en la lista de usuarios
    """
    # saco la lista de usuarios original, de donde voy a sacar la cuenta
    df_lista_usuarios_original = dataframe_problem_recomend[dataframe_problem_recomend['problem_id'] == row['problem_id']]
    lista_usuarios_original = list(df_lista_usuarios_original['list_users'])[0]
    
    # print(lista_problemas_original)
    
    # saco el problema a contar
    user = row['possible_user']
    
    #print(problem)
    
    return lista_usuarios_original.count(user)

In [74]:
# para cada usuario a "recomendar" para cada problema miro el numero de apariciones en su lista

df_separation['score'] = df_separation.apply (lambda row: getScore(row, dataframe_problem_recomend), axis=1)

print(df_separation)

        problem_id  possible_user  score
0                2            126      8
1                2             39     11
2                2            313      1
3                2             42     23
4                2           1902      2
5                2           1983     34
6                2           1959     20
7                2           1988     13
8                2           1952     19
9                2           1962     53
10               2           3497      7
11               2             46      8
12               2            152     15
13               2            221      3
14               2             88      2
15               2            194      1
16               2             90      4
17               2             99      3
18               2            155      6
19               2            542     18
20               2            896      9
21               2           1186      1
22               2            598      9
23              

In [75]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['possible_user', 'score'], ascending=False)
print(df_separation)

        problem_id  possible_user  score
1640             2           4018      1
3562             4           4018      1
5469             6           4018      1
6945             8           4018      1
9449            10           4018      1
11303           13           4018      1
13241           15           4018      1
14744           17           4018      1
16807           19           4018      1
18704           23           4018      1
20957           25           4018      1
22767           27           4018      1
24646           29           4018      1
26903           31           4018      1
28606           33           4018      1
30377           35           4018      1
32057           39           4018      1
33338           44           4018      1
35423           47           4018      1
37073           49           4018      1
39013           51           4018      1
40990           53           4018      1
42866           57           4018      1
44953           

In [76]:
print(df_separation[df_separation['score'] > 1])

        problem_id  possible_user  score
1042             2           4011      3
2922             4           4011      3
4829             6           4011      3
6675             8           4011      3
8853            10           4011      3
10641           13           4011      3
12562           15           4011      3
14471           17           4011      3
16556           19           4011      3
18431           23           4011      3
20809           25           4011      3
22516           27           4011      3
24371           29           4011      3
26755           31           4011      3
27606           33           4011      3
30110           35           4011      3
31366           39           4011      3
35275           47           4011      3
36818           49           4011      3
38759           51           4011      3
40737           53           4011      3
42578           57           4011      3
44702           60           4011      3
46553           

In [77]:
# ahora tengo que hacer un nuevo dataframe con usuario y problema, y una lista de recommendation 

# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('possible_user')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

#print(df_recommend)

# vuelvo a crear la estructura de los datos para poder trabajar con ellos
df_recommend = pd.DataFrame({'user_id':df_recommend.index.values, 'problem_id':df_recommend['problem_id'].tolist()})
print(df_recommend)

                                             problem_id  user_id
0     [2, 4, 6, 8, 13, 15, 17, 19, 23, 25, 27, 29, 3...        5
1     [4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ...        6
2     [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...        7
3     [8, 17, 23, 25, 29, 31, 33, 35, 39, 44, 47, 49...        8
4     [8, 15, 17, 19, 23, 25, 27, 29, 31, 33, 35, 39...        9
5     [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       10
6     [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       11
7     [2, 4, 6, 8, 10, 15, 17, 19, 23, 25, 27, 29, 3...       12
8     [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       15
9     [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       16
10    [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       17
11    [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       18
12    [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       22
13    [2, 4, 6, 8, 10, 15, 17, 19, 23, 25, 27, 29, 3...       24
14    [2, 4, 6, 8, 10, 15

In [78]:
def getProblems(row, df_recommend):
    """
        Funcion que copia los posibles problemas a recomendar para los usuarios a los que tengo que recomendar
    """
    
    # saco la lista de problemas 
    df_lista_problemas_original = df_recommend[df_recommend['user_id'] == row['user_id']]
    lista_problemas_original = list(df_lista_problemas_original['problem_id'])[0]
    
    return lista_problemas_original
    

In [79]:
# df_users # en este dataframe estan los usuarios a los que quiero recomendar

# ahora voy a copiar los problemas para los usuarios a los que tengo que recomendar
df_users['recommendation'] = df_users.apply (lambda row: getProblems(row, df_recommend), axis=1)

print(df_users)

                                        list_problem_id  user_id  \
11                            [119, 114, 316, 507, 506]       18   
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25   
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60   
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130   
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414   
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443   
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448   
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689   
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912   
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935   
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711   
997                      [39, 49, 465, 159, 134, 13, 2]     1893   
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952   
1024      [325, 327, 441, 468, 471, 503, 226, 10

In [80]:
# elimino la columna de los problemas, ya que no interesa
del df_users['list_problem_id']

print(df_users)

      user_id                                     recommendation
11         18  [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...
14         25  [2, 4, 6, 8, 10, 15, 17, 19, 23, 25, 27, 29, 3...
39         60  [4, 6, 8, 10, 13, 17, 19, 25, 27, 29, 31, 33, ...
90        130  [31, 47, 331, 383, 484, 488, 504, 15, 17, 19, ...
241       414  [4, 8, 17, 19, 23, 25, 27, 29, 31, 35, 39, 47,...
255       443  [4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ...
257       448  [25, 484, 504, 2, 4, 6, 8, 10, 13, 17, 23, 29,...
392       689  [4, 6, 10, 15, 17, 19, 23, 25, 27, 29, 31, 35,...
512       912  [4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ...
523       935  [25, 31, 47, 383, 4, 13, 17, 19, 29, 35, 39, 4...
933      1711  [25, 31, 331, 383, 484, 488, 504, 53, 76, 93, ...
997      1893  [4, 6, 8, 10, 15, 17, 19, 23, 25, 27, 29, 31, ...
1021     1952  [2, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ...
1024     1955  [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...
1066     2025  [2, 4, 6, 

In [81]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    return row['recommendation'][:k]

In [82]:
k = 10

#print(list(range(1, k+1)))

rango = list(range(1, k+1))

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    #print(name_column)
    # ahora saco los k mejores problemas para cada usuario
    df_users[name_column] = df_users.apply(lambda row: getKrecomFinal(row, i), axis=1)

# he incluido una columna para cada cada k recomendacion 
print(df_users)

      user_id                                     recommendation  \
11         18  [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...   
14         25  [2, 4, 6, 8, 10, 15, 17, 19, 23, 25, 27, 29, 3...   
39         60  [4, 6, 8, 10, 13, 17, 19, 25, 27, 29, 31, 33, ...   
90        130  [31, 47, 331, 383, 484, 488, 504, 15, 17, 19, ...   
241       414  [4, 8, 17, 19, 23, 25, 27, 29, 31, 35, 39, 47,...   
255       443  [4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ...   
257       448  [25, 484, 504, 2, 4, 6, 8, 10, 13, 17, 23, 29,...   
392       689  [4, 6, 10, 15, 17, 19, 23, 25, 27, 29, 31, 35,...   
512       912  [4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ...   
523       935  [25, 31, 47, 383, 4, 13, 17, 19, 29, 35, 39, 4...   
933      1711  [25, 31, 331, 383, 484, 488, 504, 53, 76, 93, ...   
997      1893  [4, 6, 8, 10, 15, 17, 19, 23, 25, 27, 29, 31, ...   
1021     1952  [2, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ...   
1024     1955  [2, 4, 6, 8, 10, 13, 15, 17, 19, 

In [83]:
# elimino las columnas que no me interesan
del df_users['recommendation']

print(df_users)

      user_id k_recommendation_1 k_recommendation_2 k_recommendation_3  \
11         18                [2]             [2, 4]          [2, 4, 6]   
14         25                [2]             [2, 4]          [2, 4, 6]   
39         60                [4]             [4, 6]          [4, 6, 8]   
90        130               [31]           [31, 47]      [31, 47, 331]   
241       414                [4]             [4, 8]         [4, 8, 17]   
255       443                [4]             [4, 6]          [4, 6, 8]   
257       448               [25]          [25, 484]     [25, 484, 504]   
392       689                [4]             [4, 6]         [4, 6, 10]   
512       912                [4]             [4, 6]          [4, 6, 8]   
523       935               [25]           [25, 31]       [25, 31, 47]   
933      1711               [25]           [25, 31]      [25, 31, 331]   
997      1893                [4]             [4, 6]          [4, 6, 8]   
1021     1952                [2]      

In [84]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
4               [241, 128, 124, 256, 404, 469, 70, 86]       18
5                    [33, 29, 254, 308, 445, 253, 166]       25
8    [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...       60
13                       [254, 253, 308, 166, 49, 445]      130
22   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...      414
26             [134, 187, 256, 251, 90, 174, 147, 438]      443
27   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...      448
34                     [327, 23, 150, 349, 336, 6, 49]      689
44   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...      912
46          [95, 383, 262, 122, 44, 81, 252, 145, 336]      935
70   [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...     1711
73                       [471, 469, 355, 95, 243, 436]     1893
78         [374, 195, 109, 436, 95, 10, 446, 340, 379]     1952
80                            [44, 213, 136, 162, 243]     1955
90             [136, 253, 187, 342, 203,

In [85]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems}

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    list_recom_problems = list()
    list_recom_problems = df_users[name_column].tolist()
    
    name_column_metric = 'recom_problems_' + str(i)
    set_df_metric[name_column_metric] = list_recom_problems

# he generado un nuevo dataframe con todas las recomendaciones (desde 1 a k) y los problemas realizados en 
# el evaluation set para cada usuario al que quiero recomendar

metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems recom_problems_1  \
0              [241, 128, 124, 256, 404, 469, 70, 86]              [2]   
1                   [33, 29, 254, 308, 445, 253, 166]              [2]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...              [4]   
3                       [254, 253, 308, 166, 49, 445]             [31]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...              [4]   
5             [134, 187, 256, 251, 90, 174, 147, 438]              [4]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...             [25]   
7                     [327, 23, 150, 349, 336, 6, 49]              [4]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...              [4]   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]             [25]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...             [25]   
11                      [471, 469, 355, 95, 243, 436]              [4]   
12        [374, 195, 109, 436, 95, 10,

In [86]:
def one_hit(row, i):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [87]:
def mrr(row, i): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row[name_column])) and (encontrado == False):
            if row[name_column][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [88]:
def precision(row, i):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row[name_column]))

In [89]:
def recall(row, i):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [90]:
def f1(row, i):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    name_column_prec = 'precision_' + str(i)
    name_column_rec = 'recall_' + str(i)
    denominador = row[name_column_prec] + row[name_column_rec]
    
    if denominador == 0:
        return 0
    else:
        return (2 * row[name_column_prec] * row[name_column_rec]) / denominador

In [91]:
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    # ahora voy a calcular una metrica para cada usuario
    metric_df[name_one_hit] = metric_df.apply(lambda row: one_hit(row, i), axis=1)
    metric_df[name_mrr] = metric_df.apply(lambda row: mrr(row, i), axis=1)
    metric_df[name_precision] = metric_df.apply(lambda row: precision(row, i), axis=1)
    metric_df[name_recall] = metric_df.apply(lambda row: recall(row, i), axis=1)
    metric_df[name_f1] = metric_df.apply(lambda row: f1(row, i), axis=1)

print(metric_df)

                                        eval_problems recom_problems_1  \
0              [241, 128, 124, 256, 404, 469, 70, 86]              [2]   
1                   [33, 29, 254, 308, 445, 253, 166]              [2]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...              [4]   
3                       [254, 253, 308, 166, 49, 445]             [31]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...              [4]   
5             [134, 187, 256, 251, 90, 174, 147, 438]              [4]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...             [25]   
7                     [327, 23, 150, 349, 336, 6, 49]              [4]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...              [4]   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]             [25]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...             [25]   
11                      [471, 469, 355, 95, 243, 436]              [4]   
12        [374, 195, 109, 436, 95, 10,

In [92]:


f = open("C:/hlocal/TFM/vot_simple_jn", 'a')

# calculo la media de las metricas
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    result_one_hit = metric_df[name_one_hit].mean()
    result_precision = metric_df[name_precision].mean()
    result_mrr = metric_df[name_mrr].mean()
    result_recall = metric_df[name_recall].mean()
    result_f1 = metric_df[name_f1].mean()

    # lo muestro por consola
    
    print(i)
    print("###########")
    print("One hit ----------")
    print(result_one_hit)
    print("Precision ----------")
    print(result_precision)
    print("Mrr  ----------")
    print(result_mrr)
    print("Recall  ----------")
    print(result_recall)
    print("F1  ----------")
    print(result_f1)
    
    
    f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 

    
f.close()    
    

1
###########
One hit ----------
0.09230769230769231
Precision ----------
0.09230769230769231
Mrr  ----------
0.09230769230769231
Recall  ----------
0.007501901559278608
F1  ----------
0.013647455546448395
2
###########
One hit ----------
0.15384615384615385
Precision ----------
0.09230769230769231
Mrr  ----------
0.12307692307692308
Recall  ----------
0.014893913008667105
F1  ----------
0.02485039115473898
3
###########
One hit ----------
0.15384615384615385
Precision ----------
0.07692307692307693
Mrr  ----------
0.12307692307692308
Recall  ----------
0.01781705998919114
F1  ----------
0.02765691312339324
4
###########
One hit ----------
0.18461538461538463
Precision ----------
0.08076923076923077
Mrr  ----------
0.13076923076923078
Recall  ----------
0.024891611121119317
F1  ----------
0.03613474559628406
5
###########
One hit ----------
0.35384615384615387
Precision ----------
0.11384615384615387
Mrr  ----------
0.1646153846153845
Recall  ----------
0.047192773248889264
F1  -------